In [ ]:
#| hide
from profane.core import *

# profane

> Not sacred

A hacky substitute for [sacred][], logging where, when and what experiment was run. It doesn't manage config variables, it is *only* for recording what happened.

It's a wrapper for [weights & biases][wandb] so I don't have to write my own terminal output capture solution. It just gets captured the same way it would be if I was looking at it in the `wandb` dashboard.

This also means that the interface is the same as using `wandb`:

```
from profane.core import init

# somewhere in your script before you start your experiment:
init(
    project='my_project'
)
```

All kwargs are passed to `wandb` but `wandb` is always run offline.

The difference from `wandb` is that when the run finishes the logs are copied to two locations:

* `local`: a local directory where **everything** captured gets saved
* `shared`: a shared directory (like a git repo) where only the metadata and terminal logs get saved

Information about every run is saved and the logs are saved, nothing is lost, and if I need to get more detail I can go look at the `local` directory and find detailed information.

To do:

* test with a little experiment
* write where the output is saved
* add the date and time to the name

[sacred]: https://github.com/IDSIA/sacred
[wandb]: https://github.com/wandb/wandb

## Install

I haven't put it on pypi yet so a dev install is recommended:

```sh
git clone git@github.com:gngdb/profane.git
cd profane
pip install -e .
```

## How to use

Before using `profane` run setup to set `local` and `shared` storage:

In [ ]:
!profane_setup --help

usage: Set up a ~/.profanerc file and register the required directories.
       [-h] [--user USER] local_storage shared_storage

positional arguments:
  local_storage   The local storage directory, everything will be stored here.
  shared_storage  The shared storage directory, only the terminal logs and
                  metadata will be stored here.

options:
  -h, --help      show this help message and exit
  --user USER     Optional: The user name to use for the shared storage
                  directory.


Then, as described above, `profane.core.init` can be used in place of [`wandb.init`](https://docs.wandb.ai/ref/python/init), all `kwargs` are passed to `wandb.init`. The only difference is the results are saved in the directories listed above.

In addition, profane kwargs may be provided prefixed by `profane_`. Currently, this includes:

* `profane_save` a list of paths to plaintext files that should be saved in the shared storage

```python
init(
    config: Union[Dict, str, None] = None,
    project: Optional[str] = None,
    entity: Optional[str] = None,
    tags: Optional[Sequence] = None,
    group: Optional[str] = None,
    name: Optional[str] = None,
    notes: Optional[str] = None,
    profane_save: Optional[Sequence] = None,
) -> Union[Run, RunDisabled, None]
```

`project` is used to name the directory in `shared` and `local` that will store whatever is logged.


## Console Script

For commands that you'd like to log but you can't edit, a console script is included:

```
profane_subprocess <any command>
```

Will use `subprocess.run` to run `<any command>` and store the result. To set the project for the current directory, you can use [`wandb init`](https://docs.wandb.ai/ref/cli/wandb-init).